In [ ]:
!pip install mlflow --quiet

import mlflow
!databricks configure --host https://community.cloud.databricks.com/

In [35]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/a.esposito39@studenti.uniba.it/Dementia")

In [36]:
import os
import random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("/datasets/drive/MyDrive/UniBa/Computer Science/Projects/Machine Learning/data")
RANDOM_SEED = 1234
random.seed(RANDOM_SEED)

In [37]:
!mkdir /datasets -p
from google.colab import drive
drive.mount('/datasets/drive', force_remount=True)

Mounted at /datasets/drive


In [38]:
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [39]:
print("Tensorflow version:", tf.__version__)
gpus = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Available GPUs:",
      len(tf.config.list_physical_devices('GPU')),
      f"({','.join(gpus)})")

Tensorflow version: 2.6.0
Available GPUs: 1 (Tesla P100-PCIE-16GB)


In [40]:
def get_model(width=128, height=128):
    """Build a convolutional neural network model based on the Zunhair et al model.
    References
    ----------
    - https://arxiv.org/abs/2007.13224
    """

    inputs = keras.Input((width, height, 1))

    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


In [41]:
# Compile model.
def train_model(x_train, y_train, x_test, y_test, epochs=10000, verbose=2, batch_size=32, run_name="run", filename="3d_image_classification.h5"):
    # Build model.
    mlflow.tensorflow.autolog()
    with mlflow.start_run(run_name=run_name):
        model = get_model(width=128, height=128)

        train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        validation_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

        #mlflow.log_param("batch_size", batch_size)
        train_dataset = (
            train_loader.shuffle(len(x_train), seed=RANDOM_SEED)
            .batch(batch_size)
            .prefetch(128)
        )
        validation_dataset = (
            validation_loader.shuffle(len(x_test), seed=RANDOM_SEED)
            .batch(batch_size)
            .prefetch(128)
        )

        initial_learning_rate = 5e-5 # from https://doi.org/10.3938/jkps.75.597
        #mlflow.log_param("initial_learning_rate", initial_learning_rate)
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
        )
        model.compile(
            loss="binary_crossentropy",
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=['acc'],
        )

        # Define callbacks.
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
            filename, save_best_only=True
        )
        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="acc", patience=35)

        # Train the model, doing validation at the end of each epoch
        history = model.fit(
            train_dataset,
            validation_data=validation_dataset,
            epochs=epochs,
            shuffle=True,
            verbose=verbose,
            callbacks=[checkpoint_cb, early_stopping_cb],
        )
        tf.keras.backend.clear_session()
        mlflow.log_artifacts(str(Path(filename).parent))

        model.load_weights(filename)
        metrics = model.evaluate(x_test, y_test)
        mlflow.log_metric("val_loss", metrics[0])
        mlflow.log_metric("val_accuracy", metrics[1])
        
        predictions = model.predict(x_test)
        matrix = tf.math.confusion_matrix(y_test, predictions)
        t_n, f_p, f_n, t_p = matrix.numpy().ravel()
        mlflow.log_metric("confusion.tn", t_n)
        mlflow.log_metric("confusion.fp", f_p)
        mlflow.log_metric("confusion.fn", f_n)
        mlflow.log_metric("confusion.tp", t_p)

    return history
    return history, metrics, matrix

In [42]:

for batchsize in [128, 64, 32, 16, 8]:
    print("Batchsize:", batchsize)
    for k in range(1, 11):
        base = DATA_DIR / f'both-folds/fold-{k}'
        xtrain = np.load(base / 'xtrain.npy')
        ytrain = np.load(base / 'ytrain.npy')
        xtest = np.load(base / 'xtest.npy')
        ytest = np.load(base / 'ytest.npy')
        tf.keras.backend.clear_session()

        results = DATA_DIR / f'both-folds/results/fold-{k}'
        results.mkdir(exist_ok=True, parents=True)
        train_model(xtrain, ytrain, xtest, ytest, batch_size=batchsize, run_name=f"batch{batchsize}-fold{k}", filename=str(results / f"model.h5"), verbose=0)

Batchsize: 128
INFO:tensorflow:Assets written to: /tmp/tmplwbag8nc/model/data/model/assets
21/21 [==============================] - 0s 6ms/step - loss: 0.0549 - acc: 0.9830
INFO:tensorflow:Assets written to: /tmp/tmpjd032wok/model/data/model/assets
21/21 [==============================] - 0s 6ms/step - loss: 0.0710 - acc: 0.9768
INFO:tensorflow:Assets written to: /tmp/tmpy54pqr30/model/data/model/assets
21/21 [==============================] - 0s 6ms/step - loss: 0.0412 - acc: 0.9830
INFO:tensorflow:Assets written to: /tmp/tmp77qqinz3/model/data/model/assets
21/21 [==============================] - 0s 7ms/step - loss: 0.0390 - acc: 0.9892
INFO:tensorflow:Assets written to: /tmp/tmpsb2_gqqc/model/data/model/assets
21/21 [==============================] - 0s 6ms/step - loss: 0.0238 - acc: 0.9923
INFO:tensorflow:Assets written to: /tmp/tmp0nnytyci/model/data/model/assets
21/21 [==============================] - 0s 6ms/step - loss: 0.0318 - acc: 0.9907
INFO:tensorflow:Assets written to: /t